# Modul #4 üçün Tapşırıqlar: 3-cü Hissə

In [ ]:
!pip install scikit-optimize

In [23]:
# Lazım olan kitabxanalar
from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, balanced_accuracy_score, recall_score, fbeta_score
from sklearn.feature_selection import SelectFromModel, SelectFdr
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import sklearn
import pickle
import skopt

In [4]:
sns.set(font_scale = 1.5, style = 'darkgrid', palette = 'bright')
pd.set_option('display.max_columns', None)
warnings.filterwarnings(action = 'ignore')
sklearn.set_config(display = 'diagram')
np.random.seed(seed = 42)

In [5]:
def generate_data(regression = True, n_observations = None, n_features = None, n_missing = None):
    if regression:
        X, Y = make_regression(n_samples = n_observations, n_features = n_features, random_state = 42)
    else:
        X, Y = make_classification(n_samples = n_observations, n_features = n_features, random_state = 42)

    column_names = [f'feature_{x}' for x in range(1, n_features + 1)]

    X = pd.DataFrame(data = X)
    Y = pd.Series(data = Y, name = 'target').to_frame()

    X.columns = column_names

    rock = ('Rock ' * 25).split()
    jazz = ('Jazz ' * 25).split()
    metal = ('Metal ' * 25).split()
    lyric = ('Lyric ' * 25).split()

    X[f'feature_{n_features + 1}'] = rock + jazz + metal + lyric

    data_frame = pd.concat(objs = [X, Y], axis = 1)

    assert data_frame.shape[0] == n_observations
    assert data_frame.shape[1] == n_features + 2

    column_names = column_names + [column for column in X.columns.tolist() if column not in column_names]

    for feature in column_names:
        data_frame.loc[data_frame.index.isin(values = data_frame[feature].sample(n = n_missing).index), feature] = np.nan

    data_frame = data_frame.sample(frac = 1.0, random_state = 42, ignore_index = True)

    return data_frame

In [6]:
# Sinifləndirmə datasetin yaradılması
df = generate_data(regression = False, n_observations = 100, n_features = 10, n_missing = 50)

# İlk beş sətrin göstərilməsi
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,target
0,NaN,NaN,NaN,NaN,-0.219101,NaN,NaN,NaN,NaN,0.235615,Lyric,0
1,NaN,NaN,NaN,0.703852,NaN,-0.727137,0.620672,-1.335344,NaN,0.177701,NaN,1
2,NaN,2.304500,1.83991,1.339702,NaN,NaN,0.444263,1.159330,NaN,-0.360966,Metal,1
3,NaN,NaN,NaN,0.211646,-0.478749,0.222134,NaN,NaN,1.255756,NaN,NaN,1
4,NaN,-0.713525,NaN,NaN,NaN,NaN,-0.525755,0.150394,-2.123896,-0.759133,NaN,1


In [7]:
# Asılı olmayan dəyişənlərin filterlənməsi
X = df.drop(columns = 'target')

# Asılı olan dəyişənin filterlənməsi
Y = df.target

# Asılı olmayan dəyişənlərin və asılı olan dəyişənin train & test setə bölünməsi
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [8]:
# LogisticRegression modelinin qurulması
algorithm = LogisticRegression(solver = 'liblinear', max_iter = 500, random_state = 42)

# Kateqorik dəyişənlər üçün borunun yaradılması
categoric_pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'most_frequent')), ('ohe', OneHotEncoder(handle_unknown = 'ignore'))])

# Numerik dəyişənlər üçün borunun yaradılması
numeric_pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'median')), ('scaler', RobustScaler())])

# Kateqorik və numerik borunun birləşdirilməsi
feature_transformer = ColumnTransformer(transformers = [('categoric_transformer', categoric_pipeline, X_train.select_dtypes(include = 'object').columns.tolist()),
                                                        ('numeric_transformer', numeric_pipeline, X_train.select_dtypes(include = 'number').columns.tolist())], n_jobs = -1)

# Sinifləndirmə borusunun yaradılması
pipe = Pipeline(steps = [('feature_transformer', feature_transformer), ('feature_selector', SelectFromModel(estimator = algorithm)), ('classifier', algorithm)])

# Boru arxitekturasının göstərilməsi
pipe

Pipeline(steps=[('feature_transformer',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('categoric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['feature_11']),
                                                 ('numeric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['feature_1', 'feature_2',
                                                   'feature_3', 'feature_4',
                                                   'feature_5', 'feature_6',
                                                   'feature_7', 'feature_8',
                                                   'feature_9',
                                                   'feature_10'])])),
                ('feature_selector',
                 SelectFromModel(estimator=LogisticRegression(max_iter=500,
                                                              random_state=42,
                                                              solver='liblinear'))),
                ('classifier',
                 LogisticRegression(max_iter=500, random_state=42,
                                    solver='liblinear'))])

In [9]:
def apply_bayesian_optimization(model = None, hyperparameters = None, n_iterations = 50, metric = None, train_features = None, train_labels = None):
    # Çarpaz validasiya texnikasının təyin olunması
    rskf = RepeatedStratifiedKFold(random_state = 42)

    # Bayesian Optimization alqoritması ilə ən optimal hiper parametrlərin tapılması
    bayesian_optimization = skopt.BayesSearchCV(estimator = model, search_spaces = hyperparameters, n_iter = n_iterations, scoring = metric, n_jobs = -1, cv = rskf, random_state = 42)
    bayesian_optimization.fit(X = train_features, y = train_labels)

    # Ən optimal hiper parametrlərdən ibarət sinifləndirmə borusunun seçilməsi
    best_estimator = bayesian_optimization.best_estimator_

    return best_estimator

In [10]:
# Ən optimal hiper parametrlərin tapılması üçün lügət data struktunun yaradılması
search_spaces = {}

# Hiper parametrlərin lügət data strukturuna daxil edilməsi
search_spaces['feature_transformer__numeric_transformer__scaler'] = skopt.space.Categorical(categories = [RobustScaler(), MinMaxScaler(), MaxAbsScaler(), StandardScaler()])
search_spaces['feature_transformer__numeric_transformer__imputer__strategy'] = skopt.space.Categorical(categories = ['mean', 'median', 'constant', 'most_frequent'])
search_spaces['feature_transformer__categoric_transformer__imputer__strategy'] = skopt.space.Categorical(categories = ['constant', 'most_frequent'])
search_spaces['feature_selector'] = skopt.space.Categorical(categories = [SelectFromModel(estimator = algorithm), SelectFdr()])
search_spaces['classifier__C'] = skopt.space.Real(low = 1e-6, high = 100)
search_spaces['classifier__tol'] = skopt.space.Real(low = 1e-6, high = 100)

# Sinifləndirmə borusunda ən optimal hiper parametrlərin tapılması
best_pipe = apply_bayesian_optimization(model = pipe, hyperparameters = search_spaces, n_iterations = 100, metric = 'accuracy', train_features = X_train, train_labels = y_train)

# Boru arxitekturasının göstərilməsi
best_pipe

Pipeline(steps=[('feature_transformer',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('categoric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['feature_11']),
                                                 ('numeric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['feature_1', 'featu...,
                                                   'feature_3', 'feature_4',
                                                   'feature_5', 'feature_6',
                                                   'feature_7', 'feature_8',
                                                   'feature_9',
                                                   'feature_10'])])),
                ('feature_selector',
                 SelectFromModel(estimator=LogisticRegression(max_iter=500,
                                                              random_state=42,
                                                              solver='liblinear'))),
                ('classifier',
                 LogisticRegression(C=1.2495676715743693, max_iter=500,
                                    random_state=42, solver='liblinear',
                                    tol=0.0006424998832159156))])

***
#### Tapşırıq #1

- **find_optimal_threshold** adlı funksiya yarat və bu funksiyaya 6 parametr ver.
1. model - Ən optimal hiper parametrlərdən ibarət sinifləndirmə borusu
2. metric - Ən optimal ehtimal sərhəddinin tapılması üçün seçilən qiymətləndirmə meyarı
3. train_features - Asılı olmayan train dəyişənləri
4. train_labels - Asılı olan train dəyişənləri
5. test_features - Asılı olmayan test dəyişənləri
6. test_labels - Asılı olan test dəyişənləri

- Daha sonra 0.1-dən 1.0-a kimi (1.0 daxildir) mərhələ dəyəri 0.01 olmaqla array data strukturu yarat.
- Daha sonra metric parametrinə əsasən verilmiş qiymətləndirmə meyarına (qiymətləndirmə meyarları accuracy, balanced_accuracy, negative_precision & positive_precision, negative_recall & positive_recall) görə şərt yarat və həmin qiymətləndirmə meyarlarını həm train həm də test seti üçün hər bir ehtimal sərhəddinə uygun olaraq hesabla.
- Daha sonra həm train həm də test seti üçün ən optimal ehtimal sərhəddini indeksləyərək array data strukturundan filterlə və hər iki ehtimal sərhəddini geri qaytar

In [11]:
def find_optimal_threshold(model, metric, train_features, train_labels, test_features, test_labels):
  probabilities = np.arange(0.1, 1.01, 0.01)


  metrics_dict = {'accuracy':'Accuracy', 'balanced_accuracy':'Balanced Accuracy',
                  'positive_precision':'Positive Precision', 'negative_precision':'Negative Precision',
                  'positive_recall':'Positive Recall', 'negative_recall':'Negative Recall'}

  if metric == 'accuracy':
      train_metrics_per_proba = [accuracy_score(y_true = train_labels, y_pred = np.where(model.predict_proba(X = train_features)[:, 1] >= proba, 1, 0)) for proba in probabilities]
      test_metrics_per_proba = [accuracy_score(y_true = test_labels, y_pred = np.where(model.predict_proba(X = test_features)[:, 1] >= proba, 1, 0)) for proba in probabilities]
  elif metric == 'balanced_accuracy':
      train_metrics_per_proba = [balanced_accuracy_score(y_true = train_labels, y_pred = np.where(model.predict_proba(X = train_features)[:, 1] >= proba, 1, 0)) for proba in probabilities]
      test_metrics_per_proba = [balanced_accuracy_score(y_true = test_labels, y_pred = np.where(model.predict_proba(X = test_features)[:, 1] >= proba, 1, 0)) for proba in probabilities]
  elif metric == 'positive_precision':
      train_metrics_per_proba = [precision_score(y_true = train_labels, y_pred = np.where(model.predict_proba(X = train_features)[:, 1] >= proba, 1, 0)) for proba in probabilities]
      test_metrics_per_proba = [precision_score(y_true = test_labels, y_pred = np.where(model.predict_proba(X = test_features)[:, 1] >= proba, 1, 0)) for proba in probabilities]
  elif metric == 'negative_precision':
      train_metrics_per_proba = [precision_score(y_true = train_labels, y_pred = np.where(model.predict_proba(X = train_features)[:, 1] >= proba, 1, 0), pos_label = 0) for proba in probabilities]
      test_metrics_per_proba = [precision_score(y_true = test_labels, y_pred = np.where(model.predict_proba(X = test_features)[:, 1] >= proba, 1, 0), pos_label = 0) for proba in probabilities]
  elif metric == 'positive_recall':
      train_metrics_per_proba = [recall_score(y_true = train_labels, y_pred = np.where(model.predict_proba(X = train_features)[:, 1] >= proba, 1, 0)) for proba in probabilities]
      test_metrics_per_proba = [recall_score(y_true = test_labels, y_pred = np.where(model.predict_proba(X = test_features)[:, 1] >= proba, 1, 0)) for proba in probabilities]
  elif metric == 'negative_recall':
      train_metrics_per_proba = [recall_score(y_true = train_labels, y_pred = np.where(model.predict_proba(X = train_features)[:, 1] >= proba, 1, 0), pos_label = 0) for proba in probabilities]
      test_metrics_per_proba = [recall_score(y_true = test_labels, y_pred = np.where(model.predict_proba(X = test_features)[:, 1] >= proba, 1, 0), pos_label = 0) for proba in probabilities]

  best_threshold_train = probabilities[np.array(object = train_metrics_per_proba).argmax()]
  best_threshold_test = probabilities[np.array(object = test_metrics_per_proba).argmax()]

  return best_threshold_train, best_threshold_test

In [12]:
# find_optimal_threshold() funksiyası ilə optimal ehtimal sərhəddinin tapılması
train_threshold, test_threshold = find_optimal_threshold(model = best_pipe, metric = 'accuracy', train_features = X_train, train_labels = y_train, test_features = X_test, test_labels = y_test)

***
#### Tapşırıq #2

- **plot_confusion_matrix** adlı funksiya yarat və bu funksiyaya 5 parametr ver.
1. model - Ən optimal hiper parametrlərdən ibarət sinifləndirmə borusu
2. train_features - Asılı olmayan train dəyişənləri
3. train_labels - Asılı olan train dəyişənləri
4. test_features - Asılı olmayan test dəyişənləri
5. test_labels - Asılı olan test dəyişənləri

Bu funksiyadan istifadə edərək confusion matrix, precision və recall ratio-nu train və test setə görə hesabla.

In [22]:
def plot_confusion_matrix(model, train_features, train_labels, test_features, test_labels):
    train_pred = model.predict(train_features)
    test_pred = model.predict(test_features)

    train_cm = confusion_matrix(train_labels, train_pred)
    test_cm = confusion_matrix(test_labels, test_pred)

    train_precision = precision_score(train_labels, train_pred)
    test_precision = precision_score(test_labels, test_pred)
    train_recall = recall_score(train_labels, train_pred)
    test_recall = recall_score(test_labels, test_pred)


    plt.figure(figsize=(8, 6))
    plt.imshow(train_cm, interpolation = 'nearest', cmap = plt.cm.Blues)
    plt.title('Confusion Matrix - Train Set')
    plt.colorbar()
    plt.xticks(np.arange(len(np.unique(train_labels))), np.unique(train_labels))
    plt.yticks(np.arange(len(np.unique(train_labels))), np.unique(train_labels))
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()


    print(f'Train Set - Precision: {train_precision:.4f}, Recall: {train_recall:.4f}')


    plt.figure(figsize = (8, 6))
    plt.imshow(test_cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix - Test Set')
    plt.colorbar()
    plt.xticks(np.arange(len(np.unique(test_labels))), np.unique(test_labels))
    plt.yticks(np.arange(len(np.unique(test_labels))), np.unique(test_labels))
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

    print(f'Test Set - Precision: {test_precision:.4f}, Recall: {test_recall:.4f}')

In [ ]:
# Funksiyanın çağırılması
plot_confusion_matrix(model = best_pipe, train_features = X_train, train_labels = y_train, test_features = X_test, test_labels = y_test)

***
### Tapşırıq #3:
- evaluate_model_performance adlı funksiya yarat və bu funksiyaya 7 parametr ver.
1. model - Ən optimal hiper parametrlərdən ibarət sinifləndirmə borusu
2. train_features - Asılı olmayan train dəyişənləri
3. train_labels - Asılı olan train dəyişənləri
4. test_features - Asılı olmayan test dəyişənləri
5. test_labels - Asılı olan test dəyişənləri
6. algorithm_name - İstifadə olunan alqoritmanın adı
7. beta - F Beta Score-un hesablanması lazım olan beta dəyəri

Bu funksiyadan istifadə edərək bu qiymətləndirmə meyarlarını test set-ə əsasən hesabla
1. Accuracy
2. Balanced Accuracy
3. Precision
4. Recall
5. F Beta (Beta = 0.5)

In [24]:
def evaluate_model_performance(model, train_features, train_labels, test_features, test_labels, algorithm_name, beta = 0.5):
    train_pred = model.predict(train_features)
    test_pred = model.predict(test_features)

    accuracy = accuracy_score(test_labels, test_pred)
    balanced_accuracy = balanced_accuracy_score(test_labels, test_pred)
    precision = precision_score(test_labels, test_pred)
    recall = recall_score(test_labels, test_pred)
    fbeta = fbeta_score(test_labels, test_pred, beta = beta)

    print(f'Algorithm: {algorithm_name}')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Balanced Accuracy: {balanced_accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F Beta ({beta}): {fbeta:.4f}')

In [25]:
# Funksiyanın çağırılması
model_summary_df = evaluate_model_performance(model = best_pipe,
                                              train_features = X_train,
                                              train_labels = y_train,
                                              test_features = X_test,
                                              test_labels = y_test,
                                              algorithm_name = 'Logistic Regression')
model_summary_df

Algorithm: Logistic Regression
Accuracy: 0.6500
Balanced Accuracy: 0.6414
Precision: 0.6250
Recall: 0.5556
F Beta (0.5): 0.6098


***
#### Tapşırıq #4

- Modeli yaddaşa pickle faylı kimi yaz.

In [26]:
# Pickle kitabxanasından istifadə edərək modelin yaddaşa yazılması
with open(file = 'saved_model.pickle', mode = 'wb') as pf:
  pickle.dump(obj = best_pipe, file = pf)